In [ ]:
# Imports comunes
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Intentamos importar tensorflow y torch cuando estén disponibles
try:
    import tensorflow as tf
    from tensorflow.keras.models import load_model as keras_load_model
    TF_AVAILABLE = True
except Exception as e:
    TF_AVAILABLE = False

try:
    import torch
    import torch.nn as nn
    import torchvision.models as models
    TORCH_AVAILABLE = True
except Exception as e:
    TORCH_AVAILABLE = False

print('TF available:', TF_AVAILABLE, ' - Torch available:', TORCH_AVAILABLE)

## 1) Cargar datos desde `../Cancer_test`
Se asume que existe la estructura simplificada con subcarpetas `benign` y `malignant`.

In [ ]:
# Ruta del test simplificado
test_dir = '../Cancer_test'
categories = ['benign', 'malignant']

X_test = []
y_test = []
filenames = []

for category in categories:
    path = os.path.join(test_dir, category)
    class_num = 0 if category == 'benign' else 1
    if os.path.exists(path):
        for fname in os.listdir(path):
            fpath = os.path.join(path, fname)
            try:
                img = cv2.imread(fpath)
                img = cv2.resize(img, (128, 128))
                X_test.append(img.astype('float32') / 255.0)  # escala para TF
                y_test.append(class_num)
                filenames.append(fname)
            except Exception as e:
                # Ignoramos archivos corruptos pero los listamos
                print('WARNING, could not read', fpath, e)

X_test = np.array(X_test)
y_test = np.array(y_test)

print('Cargadas', len(X_test), 'imágenes desde', test_dir)

## 2) Buscar y cargar modelo Keras (.h5) si existe
Buscamos en `/kaggle/input` (si se está ejecutando en Kaggle) o en el working dir.

In [ ]:
keras_model = None
h5_path = None
# Buscar en /kaggle/input recursivamente
for root, dirs, files in os.walk('/kaggle/input'):
    for f in files:
        if f.endswith('.h5'):
            h5_path = os.path.join(root, f)
            break
    if h5_path:
        break

# Si no está en /kaggle/input, buscar en el directorio actual
if not h5_path:
    for root, dirs, files in os.walk('.'):
        for f in files:
            if f.endswith('.h5'):
                h5_path = os.path.join(root, f)
                break
        if h5_path:
            break

if h5_path and TF_AVAILABLE:
    print('Cargando modelo Keras desde', h5_path)
    try:
        keras_model = keras_load_model(h5_path)
        print('Modelo Keras cargado en variable `keras_model`.')
    except Exception as e:
        print('ERROR cargando .h5:', e)
else:
    print('No se encontró .h5 o TensorFlow no disponible. h5_path=', h5_path)

### 2.1) Evaluación con el modelo Keras (si está cargado)
Se generan métricas por imagen y métricas agregadas por paciente (PS, RR, WTA).

In [ ]:
if keras_model is not None:
    print('Ejecutando predicciones Keras...')
    y_pred_probs = keras_model.predict(X_test, verbose=0)
    y_pred_classes = (y_pred_probs > 0.5).astype('int32').flatten()

    print('
--- Classification Report (Keras) ---')
    print(classification_report(y_test, y_pred_classes, target_names=['Benign', 'Malignant']))

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Benign','Malignant'], yticklabels=['Benign','Malignant'])
    plt.title('Matriz de Confusión (Keras)')
    plt.ylabel('Verdadero')
    plt.xlabel('Predicho')
    plt.show()

    # ROC
    try:
        fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(6,5))
        plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
        plt.plot([0,1],[0,1],'--')
        plt.title('ROC (Keras)')
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.legend()
        plt.show()
    except Exception as e:
        print('No se pudo calcular ROC:', e)

    # Metrics por paciente (PS, RR, WTA)
    df_results = pd.DataFrame({'filename': filenames, 'true_label': y_test, 'pred_label': y_pred_classes})
    # intentar extraer patient_id desde el nombre original (mismo heurístico del notebook)
    try:
        df_results['patient_id'] = df_results['filename'].apply(lambda x: x.split('-')[2])
    except Exception:
        df_results['patient_id'] = df_results['filename']

    patient_scores = df_results.groupby('patient_id').apply(lambda x: (x['true_label'] == x['pred_label']).mean())
    recognition_rate = patient_scores.mean()
    wta_df = df_results.groupby('patient_id').agg({
        'true_label': lambda x: x.mode()[0],
        'pred_label': lambda x: x.mode()[0]
    })
    wta_accuracy = (wta_df['true_label'] == wta_df['pred_label']).mean()

    print('
--- Métricas a nivel paciente (Keras) ---')
    print(f'Total pacientes: {len(patient_scores)}')
    print(f'Recognition Rate (RR): {recognition_rate:.2%}')
    print(f'Winner Takes All (WTA): {wta_accuracy:.2%}')
else:
    print('No hay modelo Keras cargado, se omite evaluación Keras.')

## 3) Buscar y cargar modelo PyTorch (.pth) si existe
Construimos la arquitectura ResNet18 (capa final adaptada) y cargamos los pesos si encontramos un archivo .pth en `/kaggle/input` o en el working dir.

In [ ]:
pth_path = None
for root, dirs, files in os.walk('/kaggle/input'):
    for f in files:
        if f.endswith('.pth'):
            pth_path = os.path.join(root, f)
            break
    if pth_path:
        break

if not pth_path:
    for root, dirs, files in os.walk('.'):
        for f in files:
            if f.endswith('.pth'):
                pth_path = os.path.join(root, f)
                break
        if pth_path:
            break

print('pth_path=', pth_path)
model_cancer2 = None
if pth_path and TORCH_AVAILABLE:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Creando ResNet18 en device', device)
    try:
        # Crear ResNet18 y adaptar fc
        try:
            model_cancer2 = models.resnet18(weights=None)
        except TypeError:
            model_cancer2 = models.resnet18(pretrained=False)
        model_cancer2.fc = nn.Linear(model_cancer2.fc.in_features, 2)
        state_dict = torch.load(pth_path, map_location=device)
        model_cancer2.load_state_dict(state_dict)
        model_cancer2.to(device)
        model_cancer2.eval()
        print('Modelo PyTorch cargado en `model_cancer2`')
    except Exception as e:
        print('ERROR cargando .pth:', e)
else:
    print('No se encontró .pth o PyTorch no disponible. pth_path=', pth_path)

### 3.1) Inferencia y métricas con ResNet18 (PyTorch)

In [ ]:
if model_cancer2 is not None:
    device = next(model_cancer2.parameters()).device
    print('Ejecutando inferencia PyTorch en', device)
    # Definimos preprocesado robusto (acepta uint8 [0,255] o float [0,1])
    def preprocess_resnet_custom_from_array(img_array):
        import torch
        import numpy as np
        # convertir a numpy
        arr = np.array(img_array)
        h, w = arr.shape[:2]
        target_size = 224
        scale = target_size / max(h, w)
        new_w, new_h = int(w * scale), int(h * scale)
        # manejar arrays en float [0,1] o uint8 [0,255]
        if np.issubdtype(arr.dtype, np.floating):
            img_to_resize = np.clip(arr * 255.0, 0, 255).astype('uint8')
        else:
            img_to_resize = arr.astype('uint8')
        # redimensionar (dsize = (width, height))
        img_resized = cv2.resize(img_to_resize, (new_w, new_h))
        delta_w = target_size - new_w
        delta_h = target_size - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        img_padded = cv2.copyMakeBorder(img_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
        # BGR a RGB (cv2.imread devuelve BGR)
        img_rgb = cv2.cvtColor(img_padded, cv2.COLOR_BGR2RGB)
        tensor = torch.from_numpy(img_rgb).permute(2,0,1).float() / 255.0
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
        tensor = (tensor - mean) / std
        return tensor.unsqueeze(0)

    filenames_resnet = []
    y_true_resnet = []
    y_prob_resnet = []

    with torch.no_grad():
        for i, fname in enumerate(filenames):
            # reconstruimos la ruta al archivo original en ../Cancer_test/*/fname
            found = False
            for cat in categories:
                p = os.path.join(test_dir, cat, fname)
                if os.path.exists(p):
                    img_bgr = cv2.imread(p)
                    found = True
                    break
            if not found:
                continue
            img_tensor = preprocess_resnet_custom_from_array(img_bgr)
            img_tensor = img_tensor.to(device)
            outputs = model_cancer2(img_tensor)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            malignant_prob = probs[0][1].item()
            # extraer label verdadero a partir de la carpeta donde se encontró
            true_label = 0 if os.path.dirname(p).endswith('benign') else 1
            filenames_resnet.append(fname)
            y_true_resnet.append(true_label)
            y_prob_resnet.append(malignant_prob)

    df_resnet = pd.DataFrame({'filename': filenames_resnet, 'true_label': y_true_resnet, 'prob_malignant': y_prob_resnet})
    df_resnet['pred_label'] = (df_resnet['prob_malignant'] > 0.5).astype(int)
    try:
        df_resnet['patient_id'] = df_resnet['filename'].apply(lambda x: x.split('-')[2])
    except Exception:
        df_resnet['patient_id'] = df_resnet['filename']

    print('\n--- Classification Report (ResNet PyTorch) ---')
    print(classification_report(df_resnet['true_label'], df_resnet['pred_label'], target_names=['Benign','Malignant']))

    # ROC y Confusion
    fpr, tpr, _ = roc_curve(df_resnet['true_label'], df_resnet['prob_malignant'])
    roc_auc = auc(fpr, tpr)
    print('AUC ResNet:', roc_auc)
    cm = confusion_matrix(df_resnet['true_label'], df_resnet['pred_label'])
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['Benign','Malignant'], yticklabels=['Benign','Malignant'])
    plt.title('Matriz de Confusión (ResNet)')
    plt.show()

    # Métricas por paciente
    patient_scores_rn = df_resnet.groupby('patient_id').apply(lambda x: (x['true_label'] == x['pred_label']).mean())
    rr_resnet = patient_scores_rn.mean()
    wta_df_rn = df_resnet.groupby('patient_id').agg({
        'true_label': lambda x: x.mode()[0],
        'pred_label': lambda x: x.mode()[0]
    })
    wta_resnet = (wta_df_rn['true_label'] == wta_df_rn['pred_label']).mean()
    print('\n--- Métricas a nivel paciente (ResNet) ---')
    print(f'Recognition Rate (RR): {rr_resnet:.2%}')
    print(f'Winner Takes All (WTA): {wta_resnet:.2%}')
else:
    print('No hay modelo PyTorch cargado, se omite evaluación PyTorch.')

---
Fin del notebook `model_testing.ipynb`.

Notas:
- Este notebook no modifica ficheros en disco; solo lee desde `../Cancer_test`.
- En Kaggle, carga `models.h5` y/o `.pth` a Inputs para que el notebook los encuentre en `/kaggle/input`.
